the mothod of label converting is optimised # finish

In [1]:
import numpy as np
import pandas as pd
import torch
import os

# convert sequence to matrix
def one_hot(sequence):
    nucl_list = ['A', 'T', 'C', 'G']
    nucl_dic = dict((int, nucl) for nucl, int in enumerate(nucl_list))
    _seq_data = [nucl_dic[base] for base in sequence]
    result = np.zeros((len(sequence), 4))
    for _, base in enumerate(_seq_data):
        result[_, base] = 1
    return(result)

# convert label to tensor
def label2tensor(labels):
    # open host info file
    host_info = pd.read_csv('/home/ouconstand/data/Virus_Host/non_homo_host_info.csv')
    host_dict = host_info[['Representation', 'HostClass']].drop_duplicates().set_index('Representation').to_dict(orient='dict')['HostClass']
    
    # convert accession to class taxon
    labels = [host_dict[label] for label in labels]
    
    labels_dict = dict((key, value) for value, key in enumerate(list(set(labels))))
    labels = [labels_dict[i] for i in labels]
    result = np.zeros((len(labels), len(set(labels))))
    
    for _, base in enumerate(labels):
        result[_, base] = 1
    # convert result to tensor
    result = torch.tensor(result).to(torch.float32)
    
    return result

# cut off sequence to 2k segment
def seq2onehot(sequence_dir, num):
    
    seq_label = [] # the list is including seq corresponding accession
    seq_data = [] # a sequence is converted to a list of 2k sub-seq
    n = 0
    for file in os.listdir(sequence_dir):
        
        n += 1
        
        id = file
        with open(sequence_dir + file, 'r') as f:
            seq = f.read()
            
        new_record = []
            
        for i in range(0, len(seq), 500):
            if (i + 2000) > len(seq):
                new_seq = seq[-2000:]
                new_record.append(new_seq)
                break
            new_seq = seq[i:i+2000]
            new_record.append(new_seq)
        
        seq_data.append(torch.tensor(np.array([one_hot(segment) for segment in new_record], dtype=np.float32)))

        seq_label.append(id)
        
        if n == num:
            break

    seq_label = label2tensor(seq_label)
        
    return seq_label, seq_data

In [2]:
data_path = '/home/ouconstand/data/Virus_Host/0309_seqdata/'

In [3]:
labels, train_data = seq2onehot(data_path, 499)

In [4]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # three convolutional layers and a maxpool layer
        self.convs_layers = nn.Sequential(
            nn.Conv1d(in_channels=4, out_channels=256, kernel_size=5),
            nn.ReLU(),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=7),
            nn.ReLU(),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=9),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        # it is not necessary to build a softmax layer because of the loss function including a softmax layer
        # self.pool = nn.MaxPool1d(3)
        self.fc_1 = nn.Linear(991, 512)
        self.fc_2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
 
        
        
    def forward(self, x):
        
        x = x.permute(0,2,1)
        
        x = [self.convs_layers(i) for i in x]
        count_sub = len(x)
        x = torch.cat(x, 1)
        
        x = x.view(256, int(1982/2), count_sub) # 转化变量成最后一维，然后用mean，sqeeze将其降至0维。被除数应该与maxpool层的size相同
        
        x = torch.mean(x, dim=2)
        
        x = self.fc_1(x)
        x = self.dropout(x)
        x = self.fc_2(x)
        x = self.relu(x)
        x = torch.sum(x, dim=0)
        # x = torch.mean(x, 1)
        
        # x = torch.transpose(x, 0, 1) # transpose dim
        # x = self.fc(x)
        # x = torch.transpose(x, 0, 1)
        
        # x = self.relu(x)

        return x
    
    # def show(self):
        # return self.fc 
        

In [5]:
# input (batch_size, seq_length, feature_num)
model = MyModel()
print(model)

MyModel(
  (convs_layers): Sequential(
    (0): Conv1d(4, 256, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(256, 256, kernel_size=(7,), stride=(1,))
    (3): ReLU()
    (4): Conv1d(256, 256, kernel_size=(9,), stride=(1,))
    (5): ReLU()
    (6): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_1): Linear(in_features=991, out_features=512, bias=True)
  (fc_2): Linear(in_features=512, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [75]:
batch_size = 64 # the count of sample is 64

def train_model(data, label, model, loss_fn, optimizer):
    size = len(data)
    model.train()
    
    # compute prediction error
    pred = model(data)
    loss = loss_fn(pred, label)

    # back propafation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_size % 64 == 0:
        loss, current = loss.item(), (batch_size+1) * size
        print(f'loss:{loss:>7f} [{current:>5d}/{size:>5d}]')


In [7]:
selecting = [i for i in range(499)]
selecting_train = np.random.choice(selecting, 400, replace=False)
train_number = [i for i in selecting_train]
test_number = list(set(selecting) - set(selecting_train))

In [8]:
first_train_data = [train_data[i] for i in train_number]
first_test_label = [labels[i] for i in train_number]

test_data = [train_data[i] for i in test_number]
test_label = [labels[i] for i in test_number]

In [10]:
n = 0
epoch = 0
batch_size = 100
model.train()
for input, target in zip(first_train_data, first_test_label):
    optimizer.zero_grad()
    output = model(input)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()
    n += 1
    if n % batch_size == 0:
        epoch += 1
        print('epoch {}, loss: {}'.format(epoch, loss.item()))

epoch 1, loss: 0.6931471824645996
epoch 2, loss: 0.6931471824645996
epoch 3, loss: 0.6931471824645996
epoch 4, loss: 0.6931471824645996


In [11]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(10):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print('Epoch %!d(MISSING), Loss: %!f(MISSING)' %!((MISSING)epoch, loss.item()))

In [18]:
import torch
import torch.nn  as nn
import torch.nn.functional as F

class Seq_CNN(nn.Module):
    def __init__(self) -> None:
        super(Seq_CNN, self).__init__()
        
        self.conv_layers = nn.Conv2d(1, 256, (3, 4))
        self.conv_second = nn.Conv2d(1, 256, (5, 4))
        self.conv_thirde = nn.Conv2d(1, 256, (7, 4))

    def forward(self, x):
        x = x.unsqueeze(1)
        x = F.relu(self.conv_layers(x))
    
        print(x.shape)
        x = x.squeeze(3)
        x = x.unsqueeze(1)
        x = self.conv_second(x)


        return x

In [19]:
train_model = Seq_CNN()
print(train_model)

Seq_CNN(
  (conv_layers): Conv2d(1, 256, kernel_size=(3, 4), stride=(1, 1))
  (conv_second): Conv2d(1, 256, kernel_size=(5, 4), stride=(1, 1))
  (conv_thirde): Conv2d(1, 256, kernel_size=(7, 4), stride=(1, 1))
)


In [20]:
x = train_model(train_data[0])
print(x.shape)

torch.Size([56, 256, 1998, 1])


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 28829122560 bytes. Error code 12 (Cannot allocate memory)

In [65]:
x.squeeze(3).shape

torch.Size([56, 256, 1998])

In [21]:
torch.cuda.empty_cache()